In [20]:
import pandas as pd
from r import *
import json

data = pd.read_csv("./dataset.csv")
formule = data['Equation']
print(data)

for f in range(0, 97):
    tokens = parse_equation1(formule[f])
#     fix_sorted_tokens = fix2(tokens)
#     sorted_tokens = sort_tokens(fix_sorted_tokens)

#     tree = build_tree(sorted_tokens)
#     rez = calculate_tree(tree, 2)
#     print(f,formule[f],rez )
#     # tree = build_tree(sorted_tokens)
#     # print_infix(tree)


                                  Equation  \
0                           ((x ** 4) - 6)   
1                      (((x / 8) * 2) + 1)   
2                      (((x - 1) - 3) / 5)   
3                                  (x * 5)   
4                                  (x + 2)   
..                                     ...   
93            1*x**5 + -2*x**3 + -1*x + -5   
94              3*x**5 + 2*x**4 + 1*x + -2   
95                     -4*x**5 + -1*x + -5   
96                                4*x + +1   
97   -4*x**5 + 3*x**4 + -4*x**2 + -2*x + 5   

                                                   Xs  \
0   [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...   
1   [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...   
2   [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...   
3   [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...   
4   [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...   
..                                                ...   
93  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 

In [37]:
def calcY(data):
    expression = data["Equation"]
    calculateY = []
    tokens = parse_equation1(expression)
    fix_sorted_tokens = fix2(tokens)
    sorted_tokens = sort_tokens(fix_sorted_tokens)

    tree = build_tree(sorted_tokens)
    rez = calculate_tree(tree, 2)
    print(f,formule[f],rez )
    # tree = build_tree(sorted_tokens)
    # print_infix(tree)


In [94]:
import random
from r import calculate_tree, fix2
import math
import numpy as np
def fitness_primitive(guess_eq, df, index):
    row = df.iloc[index]
    Xs = json.loads(row['Xs'])
    
    Ys = json.loads(row['Ys'])
    real_eq = row['Equation']
    
    tokens = parse_equation1(guess_eq)
    fix_tokens = fix2(tokens)
    sorted_tokens = sort_tokens(fix_tokens)
    tree = build_tree(sorted_tokens)
    fitnessVal = 0
    for i in range(len(Xs)):
        rez = calculate_tree(tree, int(Xs[i]))
        real_val = int(Ys[i])
        sub = round(np.abs(real_val - rez), 5)
        if sub == 0:
            fitnessVal += 9999
        else:
            fitnessVal += 1/sub
    return fitnessVal
        

    
    
def generate_expression():
    operators = ["+", "-", "*", "/", "**"]
    vals = [str(random.randint(1, 10)),str(random.randint(1, 10)), "x"]
    num_terms = random.randint(2, 4)
    terms = []
    x = 0
    for i in range(num_terms):
        op = random.choice(operators)
        if (x == 0 and i == num_terms - 1):
            num = "x"
            x += 1
        else:
            num = random.choice(vals)
            if (num == "x"):
                x += 1
        terms.append(op)
        terms.append(num)

    terms.pop(0)
    expression = "".join(terms)

    return expression

expression = generate_expression()
print(expression)
fitnesVal = fitness_primitive(expression, data, 1)
print(fitnesVal)

3/3**x
100012.9412778086


In [123]:
results = {}
for i in range(100):
    expression = generate_expression()
    fitness = fitness_primitive(expression, data, 1)
    
    results[expression] = fitness
    

# print(results.values())
result = dict(sorted(results.items()))
# results = sorted(results)

print(results)


{'x-8': 200007.2225552607, 'x+5+x': 2.0184849578117188, '4*x': 1.4758275580758813, 'x*6/6**6': 14.308108834024123, '7/x': 19.274100936934534, 'x*4': 1.4758275580758813, 'x**6': 99999.0175930906, '6-x': 100004.62109052889, '1*1-1/x': 20.33934306867774, 'x+x': 3.5188021590130205, 'x-7-x**7': 0.13299840163983906, '9-x*2*9': 0.34854140384328214, '2/x**10+2': 400013.9788327107, '10*x': 14.30229424947555, '8**x-8': 1.020449514578645, '10+2*10+x': 100005.40262914785, '3-2**2+x': 100006.98927150441, '10+x': 100005.40262914785, '2/3*3+x': 5.0424341407286795, '1**5-x-x': 2.423585193647883, '4/4**x': 100012.70752341872, '3**x+x': 0.4859782760793383, '5+x': 3.760224788899071, '4/4**x*9': 14.56260843782983, 'x**5-x': 1.0402360399500084, '9-10+x': 100005.40262914785, '10**6*x': 14.30229424947555, '3+3-x': 100004.62109052889, 'x**3*x': 99999.08668164181, 'x-x': 14.30229424947555, 'x/x*8-x': 100005.32173107422, '6*x': 0.9390526448473546, '3/x/x**3': 14.076789504004479, '3**x': 0.6823310277510118, '10/

In [37]:
def calcY(data):
    expression = data["Equation"]
    calculateY = []
    tokens = parse_equation1(expression)
    fix_sorted_tokens = fix2(tokens)
    sorted_tokens = sort_tokens(fix_sorted_tokens)

    tree = build_tree(sorted_tokens)
    numbers = json.loads(data['Xs'])
    for i in numbers:
        rez = calculate_tree(tree, i)
        calculateY.append(rez)
    # print(0,expression,rez )
    print(calculateY)

    return calculateY
    

In [46]:
calculate_y=calcY(data.iloc[0])

[1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 3.25, 3.5, 3.75, 4.0, 4.25, 4.5, 4.75, 5.0, 5.25, 5.5, 5.75, 6.0, 6.25, 6.5, 6.75, 7.0, 7.25, 7.5, 7.75, 8.0, 8.25, 8.5, 8.75, 9.0, 9.25, 9.5, 9.75, 10.0, 10.25, 10.5, 10.75, 11.0, 11.25, 11.5, 11.75, 12.0, 12.25, 12.5, 12.75, 13.0, 13.25, 13.5, 13.75, 14.0, 14.25, 14.5, 14.75, 15.0, 15.25, 15.5, 15.75, 16.0, 16.25, 16.5, 16.75, 17.0, 17.25, 17.5, 17.75, 18.0, 18.25, 18.5, 18.75, 19.0, 19.25, 19.5, 19.75, 20.0, 20.25, 20.5, 20.75, 21.0, 21.25, 21.5, 21.75, 22.0, 22.25, 22.5, 22.75, 23.0, 23.25, 23.5, 23.75, 24.0, 24.25, 24.5, 24.75, 25.0, 25.25, 25.5, 25.75, 26.0]


In [43]:
# def fitness_function(ga_instance, Ys,data):
#     expected_Ys = json.loads(data['Ys'])
#     errors = []
#     for i in range(len(Ys)):
#         rez = (expected_Ys[i]-Ys[i])**2
#         errors.append(rez)
#     return sum(errors)/len(errors)

In [47]:
# MSE = fitness_function(calculate_y,data.iloc[0])
# print(MSE)

0.0


In [ ]:
import pygad
ga_instance = pygad.GA(num_generations=100,
                       fitness_func=fitness_function,
                       num_parents_mating=10,
                       sol_per_pop=20,
                       mutation_type="adaptive",
                       mutation_num_genes=(3, 1))